# converter.ipynb

This jupyter notebook demonstrates how to convert DICOM (.dcm) MRI images into NifTI (.nii) format

## Install Dependencies

In [1]:
!pip install nibabel
!pip install nilearn
!pip install pydicom
!pip install tensorboardX
!pip install tqdm

## Imports

In [1]:
#General imports
import glob
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import os
import pydicom
# from pydicom.data import get_testdata_files
# from pydicom.filereader import read_dicomdir
from subprocess import call
import sys
import xml.etree.ElementTree as ET

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## Functions

In [2]:
def loadDCM(directory, sort = False):
    dataset = []

    for file in os.listdir(os.fsencode(directory)):
        filename = os.fsdecode(file)
        if filename.endswith(".dcm"):
            data = pydicom.read_file(os.path.join(directory, filename))
            #data = pydicom.dcmread(os.path.join(directory, filename))
            dataset.append(data)
    
    if sort:
        #dataset = sorted(dataset, key=lambda x: x.SliceLocation, reverse=True)
        dataset = sorted(dataset, key=lambda x: x.InstanceNumber, reverse=True)
            
    return dataset

def displayDCMInfo(dataset, start = 0, end = -1, verbose = False):
    
    if verbose:
        for i in range(start, end):
            print(dataset[i])
            print()
    else:
        #Print common attributes once
        with dataset[0] as data:
            print("Patient id..............:", data.PatientID)
            print("Modality................:", data.Modality)
            print("Study Description.......:", data.StudyDescription)
            print("Study Date..............:", data.StudyDate)
            print("Series Description......:", data.SeriesDescription)
            print("Series Instance UID.....:", data.SeriesInstanceUID)
            print("Frame of Reference UID..:", data.FrameOfReferenceUID)
            if 'PixelData' in data:
                rows = int(data.Rows)
                cols = int(data.Columns)
                print("Image size.......: {rows:d} x {cols:d}, {size:d} bytes".format(
                    rows=rows, cols=cols, size=len(data.PixelData)))
                if 'PixelSpacing' in data:
                    print("Pixel spacing....:", data.PixelSpacing)
            print()
        
        for i in range(start, end):
            data = dataset[i]
            print("SOP Instance UID........:", data.SOPInstanceUID)
            print("Instance Number.........:", data.InstanceNumber)
            if 'SliceLocation' in dataset[i]:
                print("Image Position.....:", data.ImagePositionPatient)
                print("Image Orientation..:", data.ImageOrientationPatient)
                print("Slice Location.....:", data.SliceLocation)
            print()

        
#Extract 3D Pixel Array
def extractPixelMatrix(dataset):
    pixels = list(map(lambda data: data.pixel_array, dataset))
    return np.array(pixels)

def saveAsNifti(data, directory, patient):
    os.makedirs(directory)
    data = nib.Nifti1Image(data, affine=np.eye(4))
    nib.save(data, os.path.join(directory, "volume-" + patient + ".nii"))

In [30]:
#Load data
src_dir = 'data/raw/sag/dicom/1575473977/'
dest_dir = 'data/raw/sag/dicom_converted/1575473977/'
dataset = loadDCM(src_dir, sort = True) 

print("Num of DCM images loaded:", len(dataset), end='\n\n')

#Extract 3D Pixel Array
dataset_pixels = extractPixelMatrix(dataset)
print("3D image dimension:", dataset_pixels.shape)


Num of DCM images loaded: 1040

3D image dimension: (1040, 256, 256)


In [28]:
#Display metadata of the first 5 frames
displayDCMInfo(dataset, start=0, end=15, verbose=False)

Patient id..............: 1575473977
Modality................: MR
Study Description.......: MR BREAST BILATERAL
Study Date..............: 19991003
Series Description......: SAG POST
Series Instance UID.....: 9999.150304060546306973224365164370887611217
Frame of Reference UID..: 9999.242136065787416660593293942442553187460
Image size.......: 256 x 256, 131072 bytes
Pixel spacing....: ['1.0547', '1.0547']

SOP Instance UID........: 9999.212820969752531162755733812368269136073
Instance Number.........: 1040
Image Position.....: ['-218.738', '-126.966', '135.558']
Image Orientation..: ['-0', '1', '0', '-0', '-0', '-1']
Slice Location.....: 218.7380219

SOP Instance UID........: 9999.1986617825982244649676385659190797263
Instance Number.........: 1039
Image Position.....: ['-216.738', '-126.966', '135.558']
Image Orientation..: ['-0', '1', '0', '-0', '-0', '-1']
Slice Location.....: 216.7380219

SOP Instance UID........: 9999.74239421763104661820240935570927963971
Instance Number.........: 

In [32]:
#Save as .nii file
saveAsNifti(dataset_pixels, dest_dir, '1575473977')

In [39]:
#Convenience function to do the entire conversion
def convert(patient):
    src_dir = 'data/raw/sag/dicom/' + patient + '/'
    dest_dir = 'data/raw/sag/dicom_converted/' + patient + '/'
    dataset = loadDCM(src_dir, sort = True) 

    #Extract 3D Pixel Array
    dataset_pixels = extractPixelMatrix(dataset)
    print("3D image dimension:", dataset_pixels.shape)
    
    #Transpose (for SAG only)
    dataset_pixels = dataset_pixels.transpose(0, 2, 1)
    
    #Save as .nii file
    saveAsNifti(dataset_pixels, dest_dir, patient)

In [40]:
directory = 'data/raw/sag/dicom/'
for file in os.listdir(os.fsencode(directory)):
    patient = os.fsdecode(file)
    print('Converting:', patient)
    convert(patient)

Converting: 2493250475
3D image dimension: (460, 512, 512)
Converting: 1688915074
3D image dimension: (1000, 256, 256)
Converting: 1575473977
3D image dimension: (1040, 256, 256)
Converting: 2171182440
3D image dimension: (156, 512, 512)
Converting: 3125791139
3D image dimension: (384, 256, 256)
Converting: 2868733460
3D image dimension: (140, 512, 512)
Converting: 3064160959
3D image dimension: (288, 512, 512)
Converting: 4707565301
3D image dimension: (222, 512, 512)


In [4]:
#Special case: multi-sweep MRI
patient='2493250475'
src_dir = 'data/raw/sag/dicom/' + patient + '/'
dest_dir = 'data/raw/sag/dicom_converted/' + patient + '_'
dataset = loadDCM(src_dir, sort = True) 

#Extract 3D Pixel Array
dataset_pixels = extractPixelMatrix(dataset)

#Transpose (for SAG only)
dataset_pixels = dataset_pixels.transpose(0, 2, 1)

print("3D image dimension:", dataset_pixels.shape)
D, H, W = dataset_pixels.shape
#Save as .nii file
for i in range(5):
    s = dataset_pixels[D//5*i:D//5*(i+1),:,:]
    saveAsNifti(s, dest_dir + str(i) + '/', patient)

3D image dimension: (460, 512, 512)
